In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
# -----------------------------
# 1. Load & aggregate 10 seeds
# -----------------------------
all_dicts = []
seeds = [8302, 2766, 257, 7600, 6657, 8226, 6841, 4908, 1321, 7857]
num_seeds = len(seeds)

for seed in seeds:
    # Replace the filename pattern as appropriate
    file_path = f"seed-{seed}-transfer-two-obj.npy"
    data = np.load(file_path, allow_pickle=True).item()
    all_dicts.append(data)

# -----------------------------
# 2. Average the nested dicts
# -----------------------------
# We'll create an averaged dictionary (average_dict)
average_dict = {}

for seed_dict in all_dicts:
    for target_obj_name, obj_task_dict in seed_dict.items():
        if target_obj_name not in average_dict:
            average_dict[target_obj_name] = {}
        for obj_name, affecting_tasks_dict in obj_task_dict.items():
            if obj_name not in average_dict[target_obj_name]:
                average_dict[target_obj_name][obj_name] = {}
            for affecting_task, diff_value in affecting_tasks_dict.items():
                if affecting_task not in average_dict[target_obj_name][obj_name]:
                    average_dict[target_obj_name][obj_name][affecting_task] = 0.0
                # Accumulate the difference value
                average_dict[target_obj_name][obj_name][affecting_task] += diff_value * 100 # (cm)

# Now divide by the number of seeds to get the average
for target_obj_name, obj_task_dict in average_dict.items():
    for obj_name, affecting_tasks_dict in obj_task_dict.items():
        for affecting_task in affecting_tasks_dict:
            average_dict[target_obj_name][obj_name][affecting_task] /= num_seeds

# -----------------------------
# 3. Prepare for visualization
# -----------------------------
# Suppose we know there are exactly 3 tasks (any labels you have):
task_row = ["sphere", "cube", "ver-cylinder", "hor-cylinder", "ver-prism", "hor-prism"]
task_column = ["push", "hit"]

# We'll gather all objects from the averaged dictionary
target_objects_list = list(average_dict.keys())
num_target_objects = len(target_objects_list)

# We'll gather all off-diagonal values to find global min and max
all_values = []
for target_obj_name in target_objects_list:
    for obj_name in task_row:
        for affecting_task in task_column:
            val = average_dict[target_obj_name][obj_name].get(affecting_task, 0.0)
            all_values.append(val)

# Compute global min and max to ensure a shared color scale
global_min = min(all_values) if all_values else 0.0
global_max = max(all_values) if all_values else 1.0

# -----------------------------
# 4. Plot each object as 3x3
# -----------------------------
fig, axes = plt.subplots(
    1, num_target_objects,
    figsize=(20, 3),  # Adjust figure size to fit subplots
    squeeze=False
)

for idx, target_obj_name in enumerate(target_objects_list):
    ax = axes[0, idx]  # We used a single row: row=0, column=idx
    
    # Initialize a 3x3 matrix for this object
    matrix = np.zeros((6, 2), dtype=float)
    
    # Fill the matrix:
    #   rows => affected task (i)
    #   cols => affecting task (j)
    #
    # The diagonal is 0 by definition, so if row == col: 0
    # Otherwise, use the difference in average_dict
    for i, obj_name in enumerate(task_row):
        for j, affecting_task in enumerate(task_column):
            matrix[i, j] = average_dict[target_obj_name][obj_name].get(affecting_task, 0.0)
    
    a = sns.heatmap(
        matrix, 
        ax=ax, 
        annot=True, 
        fmt=".4f", 
        cmap="viridis", 
        vmin=global_min,   # use global min
        vmax=global_max,   # use global max
        cbar=False,        # disable individual subplot colorbar
        xticklabels=task_column, 
        yticklabels=task_row,
        annot_kws={'size': 14},   # Increase annotation (cell text) size
    )
    
     # Rotate X labels if desired
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

    ax.set_title(f"{target_obj_name}", fontsize=16)
    ax.tick_params(axis='both', labelsize=12)
    if idx != 0:
        ax.set_yticklabels([])

sm = plt.cm.ScalarMappable(
    cmap="viridis",
    norm=plt.Normalize(vmin=global_min, vmax=global_max)
)
sm.set_array([])  # Required for older matplotlib versions

# Place the colorbar below all subplots (horizontal)
cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),      # all subplot axes
    orientation='horizontal',
    fraction=0.05,                 # thickness as a fraction of the plot
    pad=0.3,                       # space between subplots & colorbar
    aspect=30                      # aspect ratio can help increase length
)
n_ticks = 5
ticks = np.linspace(global_min, global_max, n_ticks)
cbar.set_ticks(ticks)
# Adjust tick font size
cbar.ax.tick_params(labelsize=12)

# plt.tight_layout()
plt.savefig("two-obj-transfer.png", bbox_inches='tight')
plt.show()
